# Processing Mirage Data

In [1]:
import csv
import os
import json

def convert_file(filename, to_file):
    fields = ['five-tuple', 'timestamp','iat', 'packet_size']
    with open(filename) as f:
        d = json.load(f)
        with open(to_file, 'w+',newline='') as csvfile:  
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(fields)  
            for _,key in enumerate(d):
                length = len(d[key]['packet_data']['timestamp'])
                for i in range(length):
                    csvwriter.writerow([key, d[key]['packet_data']['timestamp'][i], d[key]['packet_data']['iat'][i], d[key]['packet_data']['IP_packet_bytes'][i]])

def process_files(directory_path, new_dir_path):
    # List all files in the directory
    files = os.listdir(directory_path)

    i = 0
    # Iterate through each file and call the function
    for file in files:
        file_path = os.path.join(directory_path, file)

        # Check if it's a file (not a directory)
        if os.path.isfile(file_path):
            convert_file(file_path, os.path.join(new_dir_path, str(i)+".csv"))
            i += 1

In [2]:
#process_files(r"data\MIRAGE-COVID-CCMA-2022\MIRAGE-COVID-CCMA-2022\Raw_JSON\Teams\Teams", r"data\teams_data\\")

# Processing Raw PCAP
This will be indiscriminate towards flow.

In [1]:
from scapy.all import PcapNgReader, UDP, TCP, IP
import csv

def get_five_tuple(pkt):
    if IP in pkt:
        src_ip = pkt[IP].src
        dst_ip = pkt[IP].dst

        if TCP in pkt:
            src_port = pkt[TCP].sport
            dst_port = pkt[TCP].dport
            protocol = "TCP"
        elif UDP in pkt:
            src_port = pkt[UDP].sport
            dst_port = pkt[UDP].dport
            protocol = "UDP"
        else:
            src_port = dst_port = protocol = None

        return ','.join([str(src_ip), str(dst_ip),str(src_port),str(dst_port),str(protocol)])
    return None

def filter_udp_packets(pcapng_file, to_file, label):
    fields = ['five-tuple', 'timestamp','packet_size', 'label']
    csvfile = open(to_file, 'w', newline='')
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)  
    with PcapNgReader(pcapng_file) as pcap_reader:
        for pkt in pcap_reader:
            if UDP in pkt:
                csvwriter.writerow([get_five_tuple(pkt), pkt.time, len(pkt), label])

In [2]:
#filter_udp_packets(r"data\pluto\teams-audio1.pcap", r"data\pluto\teams-audio1.csv", -1)

In [26]:
with open(r'data\pluto\logs.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader, None)
    for file_base, label in reader:
        filter_udp_packets(f"data\\pluto\\{file_base}.pcap", f"data\\pluto\\{file_base}.csv", label)

In [ ]:
import pandas as pd

def calc_interarrival():
    with open(r'data\pluto\logs.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader, None)
        for file_base, label in reader:
            

## Pluto V2
Take a bidirectional flow and split it into ingress egress

In [12]:
import csv

def create_csv_file(pcapng_file, to_file, label):
    fields = ['five-tuple', 'timestamp','packet_size', 'label']
    csvfile = open(to_file, 'w', newline='')
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)  
    with PcapNgReader(pcapng_file) as pcap_reader:
        for pkt in pcap_reader:
            csvwriter.writerow([get_five_tuple(pkt), pkt.time, len(pkt), label])
    csvfile.close()

def parse_file_list(filename):
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for file_base, label in reader:
            create_csv_file(f"data\\pluto\\new-bidirectional\\{file_base}.pcapng", f"data\\pluto\\new-bidirectional\\{file_base}.csv", label)
            print("finished " + file_base)

In [ ]:
parse_file_list(r'data\pluto\new-bidirectional\files.txt')

finished AmazonPrime10
finished AmazonPrime11
finished AmazonPrime12
finished TeamsAudio10
